# Vehicle Detection and Tracking

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time

%matplotlib inline

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:9999;
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
import sys

def print_progress(iteration, total):
    str_format = "{0:.0f}"
    percents = str_format.format(100 * (iteration / float(total)))
    filled_length = int(round(100 * iteration / float(total)))
    bar = '█' * filled_length + '-' * (100 - filled_length)

    sys.stdout.write('\r |%s| %s%%' % (bar, percents)),

    if iteration == total:
        sys.stdout.write('\n')
    sys.stdout.flush()

## Training a Classifier

### Reading the labeled data

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

vehicles = []
non_vehicles = []
vehicles_paths = glob.glob('labeled_data/vehicles/*/*.png')
non_vehicles_paths = glob.glob('labeled_data/non-vehicles/*/*.png')

def append_labeled_data(items, data_type):
    for item in items:
        data_type.append(mpimg.imread(item))
        print_progress(len(data_type), len(items))
        
    
append_labeled_data(vehicles_paths, vehicles)
append_labeled_data(non_vehicles_paths, non_vehicles)

# convert vehicles and non vehicles to numpy arrays
vehicles = np.asarray(vehicles)
non_vehicles = np.asarray(non_vehicles)

print('Data imported.')

print('Vehicle image count: {}'.format(vehicles.shape[0]))
print('Non Vehicle image count: {}'.format(non_vehicles.shape[0]))



 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100%
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100%
Data imported.
Vehicle image count: 8792
Non Vehicle image count: 8968


### Extracting Features 

In [ ]:
from skimage.feature import hog

class ExtractFeatures(object):
    """Abstracts the multiple features that are extracted from each frame/image"""
    
    def __init__(self, img):    
        # convert img to YCrCb
        self.img = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        self.height = self.img.shape[0]
        self.width  = self.img.shape[1]
        self.depth  = self.img.shape[2]
        self.pix_per_cell = 8
        self.features_hog = []
        self._init_hog()
        
    def _init_hog(self):
        for channel in range(self.depth):
            hog_feature = hog(self.img[:, :, channel], 
                              orientations=10, 
                              pixels_per_cell=(self.pix_per_cell, self.pix_per_cell),
                              cells_per_block=(2, 2), 
                              transform_sqrt=True, 
                              visualise=False,
                              feature_vector=False)
            self.features_hog.append(hog_feature)
        # Convert to np array
        self.features_hog = np.asarray(self.features_hog)
        
    def _hog(self, x, y, s):
        _x = max((x // self.pix_per_cell) - 1, 0)
        _y = max((y // self.pix_per_cell) - 1, 0)
        _s = (s // self.pix_per_cell) - 1
        
        if (_x + _s) > self.features_hog.shape[2]:
            _x = self.features_hog.shape[2] - _s
            
        if (_y + _s) > self.features_hog.shape[1]:
            _y = self.features_hog.shape[1] - _s
        
        hog_region_features = np.ravel(self.features_hog[:, _y:_y + _s, _x:_x + _s, :, :, :])
        
        return hog_region_features
    
    def _bin_spatial(self, img, size=(32, 32)):
        spacial_vector = cv2.resize(img, size).ravel()
        
        return spacial_vector
        
    def _color_hist(self, img, nbins=32, bins_range=(0, 256)):
        # Compute the histogram of the color channels separately
        channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
        channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
        channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
        # Concatenate the histograms into a single feature vector
        hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
        # Return the individual histograms, bin_centers and feature vector
        return hist_features
    
    def features(self, x=0, y=0, s=64):
        """Returns a vector of the concatenated features"""
        features = []
        # Add spactial features
        spatial = self._bin_spatial(self.img[y:y + s, x:x + s, :])
        features.append(spatial)
        # Add HOG features 
        hog = self._hog(x, y, s)
        features.append(hog)
        # Add Histogram features
        hist = self._color_hist(self.img[y:y + s, x:x + s, :])
        features.append(hist)
        # Finally concatente them all and return
        return np.concatenate(features)
        

def extract_features(items, data_features):
    for item in items:
        features = ExtractFeatures(item).features()
        data_features.append(features)
        print_progress(len(data_features), len(items))

vehicle_features = []
non_vehicle_features = []

extract_features(vehicles, vehicle_features)
extract_features(non_vehicles, non_vehicle_features)

X = np.vstack((vehicle_features, non_vehicle_features)).astype(np.float64)

 |----------------------------------------------------------------------------------------------------| 0%

/Users/ricardo/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100%
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100%
